In [0]:
from langchain_databricks import ChatDatabricks

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.tools import Tool, BaseTool
from langchain_core.output_parsers import StrOutputParser

In [0]:
llm = ChatDatabricks(endpoint="databricks-meta-llama-3-1-70b-instruct", max_tokens=4000)

In [0]:

feedback_prompt = PromptTemplate(
    template="""Given the following question {input}.
    Provide training feedbacks for the mentioned customer support agent. 
    You must ask sql_agent to get the relevant qa_feedback_summary column from qa_table for the mentioned customer support agent and then summarize it. 
    Write a detailed response based on the qa_feedback_summary column from qa_table only. 
    You should NOT generate your own data.
    You should NOT assume any data or returned data.
    You can end with a training plan for the customer support agent to improve the weakest area. 
    When confronted with choices, make a decision yourself with reasoning.
    """,
    input_variables=["input"],
)


llm_chain = LLMChain(llm=llm, prompt=feedback_prompt, verbose=False)

llm_chain1 = (
    feedback_prompt
    | llm
    | StrOutputParser()
)

In [0]:
feedback_tool = Tool(
    name="feedback_agent",
    func=llm_chain.run,
    description="useful for providing training feedbacks based on qa_feedback_summary from sql_agent",
    handle_tool_error = True,
    verbose = False, 
    error_message = "Error running feedback tool",
)